<a href="https://colab.research.google.com/github/Vaish181204/Suvidha-Foundation/blob/main/Qwen2_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes accelerate transformers peft evaluate rouge_score bert-score

In [2]:
# Install Unsloth and metrics libraries
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q -U evaluate rouge_score bert-score nltk pandas openpyxl

  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
  Attempting uninstall: transformers
    Found existing installation: transformers 5.1.0
    Uninstalling transformers-5.1.0:
      Successfully uninstalled transformers-5.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.0 which is incompatible.


In [3]:
# 1. Preferred installation method to avoid colab-new errors
!pip install unsloth
# 2. Force install dependencies to ensure compatibility
!pip install --no-deps xformers trl peft accelerate bitsandbytes
# 3. Required for metrics
!pip install -q -U evaluate rouge_score bert-score nltk

In [4]:
from unsloth import FastLanguageModel
import torch

model_id = "unsloth/Qwen2-7B-Instruct-bnb-4bit" # Optimized checkpoint

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048, # Supports 128k, but 2k is safer for T4 memory
    dtype = None,
    load_in_4bit = True,
)

# Apply LoRA adapters for "From-scratch" training
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)
print("SUCCESS: Qwen2-7B adapters attached!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


SUCCESS: Qwen2-7B adapters attached!


In [6]:
import pandas as pd
from datasets import Dataset

# Load and split
df = pd.read_excel("/content/NewsSumm_Final_Cleaned.xlsx")
df = df.map(lambda x: str(x) if pd.notnull(x) else x)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1, seed=42)

# ChatML format for Qwen2
qwen_prompt = """<|im_start|>system
Summarize the following article.<|im_end|>
<|im_start|>user
{}<|im_end|>
<|im_start|>assistant
{}<|im_end|>"""

def format_prompts(examples):
    texts = [qwen_prompt.format(a, s) for a, s in zip(examples["article_text"], examples["human_summary"])]
    return { "text" : texts }

tokenized_dataset = dataset.map(format_prompts, batched = True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_dataset["train"],
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 3, # 3 Epochs for benchmark accuracy
        learning_rate = 2e-4,
        fp16 = True,
        optim = "adamw_8bit",
        logging_steps = 10,
        output_dir = "qwen2_outputs",
        report_to = "none", # Disables wandb prompt
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/900 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 3 | Total steps = 339
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Step,Training Loss
10,1.946900
20,1.845300
30,1.867600
40,1.856200
50,1.815400
60,1.831500
70,1.834100
80,1.838700
90,1.948200
100,1.822100


TrainOutput(global_step=339, training_loss=1.7069766739476746, metrics={'train_runtime': 3856.7474, 'train_samples_per_second': 0.7, 'train_steps_per_second': 0.088, 'total_flos': 8.290887324582912e+16, 'train_loss': 1.7069766739476746, 'epoch': 3.0})

In [8]:
import evaluate
from bert_score import score
rouge = evaluate.load("rouge")

def run_evaluation():
    FastLanguageModel.for_inference(model)
    preds, refs = [], list(dataset["test"]["human_summary"])

    print("Generating summaries for metrics...")
    for article in dataset["test"]["article_text"]:
        inputs = tokenizer([qwen_prompt.format(article, "")], return_tensors = "pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens = 128)
        decoded = tokenizer.batch_decode(outputs)[0]
        # Extract only the summary
        summary = decoded.split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "").strip()
        preds.append(summary)

    # Calculate Results
    r = rouge.compute(predictions=preds, references=refs)
    P, R, F1 = score(preds, refs, lang="en", verbose=False)

    print(f"\nQwen2-7B Results:\nROUGE-1: {r['rouge1']:.4f}\nROUGE-2: {r['rouge2']:.4f}\nROUGE-L: {r['rougeL']:.4f}\nBERTScore: {F1.mean().item():.4f}")

run_evaluation()

Generating summaries for metrics...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Qwen2-7B Results:
ROUGE-1: 0.0042
ROUGE-2: 0.0002
ROUGE-L: 0.0022
BERTScore: 0.7284
